In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install fastai=="1.0.61"
!pip install torchsummary

     |████████████████████████████████| 239 kB 785 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19191 sha256=e682f350b00a2efe3c6fb1ffc875b8705d47d4d098e2b75ebda9d38be39d82b0
  Stored in directory: /root/.cache/pip/wheels/df/99/da/c34f202dc8fd1dffd35e0ecf1a7d7f8374ca05fbcbaf974b83
Successfully built nvidia-ml-py3
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: fastai
    Found existing installation: fastai 2.5.3
    Uninstalling fastai-2.5.3:
      Successfully uninstalled fastai-2.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
# from google.colab import drive
# drive.mount('/content/drive')
from glob import glob
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary
from fastai.metrics import accuracy
from fastai.vision import *
import numpy as np
!pip install fft-conv-pytorch
from fft_conv_pytorch import fft_conv, FFTConv2d

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = "../input/grains/grain_dataset"
np.random.seed(41)
data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(256,256), bs=16, num_workers=4).normalize()

In [ ]:
print (data.classes, data.c, len(data.train_ds), len(data.valid_ds))
print (data.train_ds.classes)
print (data.valid_ds.classes)

['anadenanthera', 'arecaceae', 'arrabidaea', 'cecropia', 'chromolaena', 'combretum', 'croton', 'dipteryx', 'eucalipto', 'faramea', 'hyptis', 'mabea', 'matayba', 'mimosa', 'mycria', 'protium', 'qualea', 'schinus', 'senegalia', 'serjania', 'syagrus', 'tridax', 'urochloa'] 23 632 158
['anadenanthera', 'arecaceae', 'arrabidaea', 'cecropia', 'chromolaena', 'combretum', 'croton', 'dipteryx', 'eucalipto', 'faramea', 'hyptis', 'mabea', 'matayba', 'mimosa', 'mycria', 'protium', 'qualea', 'schinus', 'senegalia', 'serjania', 'syagrus', 'tridax', 'urochloa']
['anadenanthera', 'arecaceae', 'arrabidaea', 'cecropia', 'chromolaena', 'combretum', 'croton', 'dipteryx', 'eucalipto', 'faramea', 'hyptis', 'mabea', 'matayba', 'mimosa', 'mycria', 'protium', 'qualea', 'schinus', 'senegalia', 'serjania', 'syagrus', 'tridax', 'urochloa']


In [ ]:
def fft_conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        FFTConv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

def conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

def triple_conv(ni, nf):
    return nn.Sequential(
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),  
        conv_block(ni, nf)
    )
def fft_triple_conv(ni, nf,size):
    return nn.Sequential(
        fft_conv_block(ni, nf,size),
        conv_block(nf, ni, size=1),  
        fft_conv_block(ni, nf, size)
    )
def maxpooling():
    return nn.MaxPool2d(2, stride=2)

In [ ]:
fft_model5 = nn.Sequential(
    fft_conv_block(3, 8, 21),
    maxpooling(),
    fft_conv_block(8, 16, 17),
    maxpooling(),
    triple_conv(16,32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_block(256,23),
    nn.Flatten(),
    nn.Linear(3887, 23)
)
fft_learn5= Learner(data, fft_model5, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(fft_learn5.summary())
fft_learn5.fit_one_cycle(100, max_lr=5e-3)

Sequential
Layer (type)         Output Shape         Param #    Trainable 
_FFTConv             [8, 256, 256]        10,584     True      
______________________________________________________________________
BatchNorm2d          [8, 256, 256]        16         True      
______________________________________________________________________
LeakyReLU            [8, 256, 256]        0          False     
______________________________________________________________________
MaxPool2d            [8, 128, 128]        0          False     
______________________________________________________________________
_FFTConv             [16, 128, 128]       36,992     True      
______________________________________________________________________
BatchNorm2d          [16, 128, 128]       32         True      
______________________________________________________________________
LeakyReLU            [16, 128, 128]       0          False     
___________________________________________________

epoch,train_loss,valid_loss,accuracy,time
0,2.850605,2.899842,0.132911,00:07
1,2.330956,1.640710,0.443038,00:07
2,1.885574,1.252272,0.588608,00:08
3,1.629741,1.620422,0.518987,00:08
4,1.416191,1.301309,0.544304,00:07
5,1.336500,2.267261,0.360759,00:07
6,1.205180,1.700374,0.506329,00:08
7,1.218699,2.829433,0.405063,00:07
8,1.318785,5.922305,0.196203,00:07
9,1.294492,3.798304,0.341772,00:08
